In [1]:
#!wget -O scibert_uncased.tar https://github.com/naver/biobert-pretrained/releases/download/v1.1-pubmed/biobert_v1.1_pubmed.tar.gz

In [2]:
#!tar -xvf scibert_uncased.tar

In [3]:
"""import torch
from transformers import BertTokenizer, BertModel
import argparse
import logging

from transformers import BertConfig, BertForPreTraining, load_tf_weights_in_bert

def convert_tf_checkpoint_to_pytorch(tf_checkpoint_path, bert_config_file, pytorch_dump_path):
    # Initialise PyTorch model
    config = BertConfig.from_json_file(bert_config_file)
    print("Building PyTorch model from configuration: {}".format(str(config)))
    model = BertForPreTraining(config)

    # Load weights from tf checkpoint
    load_tf_weights_in_bert(model, config, tf_checkpoint_path)

    # Save pytorch-model
    print("Save PyTorch model to {}".format(pytorch_dump_path))
    torch.save(model.state_dict(), pytorch_dump_path)

convert_tf_checkpoint_to_pytorch("biobert_v1.1_pubmed/model.ckpt-1000000", "biobert_v1.1_pubmed/bert_config.json", "biobert_v1.1_pubmed/pytorch_model.bin")
"""

'import torch\nfrom transformers import BertTokenizer, BertModel\nimport argparse\nimport logging\n\nfrom transformers import BertConfig, BertForPreTraining, load_tf_weights_in_bert\n\ndef convert_tf_checkpoint_to_pytorch(tf_checkpoint_path, bert_config_file, pytorch_dump_path):\n    # Initialise PyTorch model\n    config = BertConfig.from_json_file(bert_config_file)\n    print("Building PyTorch model from configuration: {}".format(str(config)))\n    model = BertForPreTraining(config)\n\n    # Load weights from tf checkpoint\n    load_tf_weights_in_bert(model, config, tf_checkpoint_path)\n\n    # Save pytorch-model\n    print("Save PyTorch model to {}".format(pytorch_dump_path))\n    torch.save(model.state_dict(), pytorch_dump_path)\n\nconvert_tf_checkpoint_to_pytorch("biobert_v1.1_pubmed/model.ckpt-1000000", "biobert_v1.1_pubmed/bert_config.json", "biobert_v1.1_pubmed/pytorch_model.bin")\n'

In [4]:
"""!ls biobert_v1.1_pubmed
!mv biobert_v1.1_pubmed/bert_config.json biobert_v1.1_pubmed/config.json
!ls biobert_v1.1_pubmed"""

'!ls biobert_v1.1_pubmed\n!mv biobert_v1.1_pubmed/bert_config.json biobert_v1.1_pubmed/config.json\n!ls biobert_v1.1_pubmed'

In [5]:
import os
import json
import pandas as pd
import numpy as np
from transformers import BertModel,BertTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
from tqdm import tqdm, trange
import copy

I0505 14:51:39.422375 47349498433216 file_utils.py:41] PyTorch version 1.2.0 available.
I0505 14:51:43.623143 47349498433216 file_utils.py:57] TensorFlow version 2.1.0 available.


In [6]:
!nvidia-smi

Tue May  5 14:51:45 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:18:00.0 Off |                    0 |
| N/A   46C    P0    59W / 300W |      0MiB / 32510MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [7]:
from create_processed_data import *

In [8]:
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForTokenClassification

In [9]:
print('Imported all libraries')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print('Loaded device:',device)

Imported all libraries
Loaded device: cuda


In [10]:
device

device(type='cuda')

In [ ]:
torch.cuda.get_device_name(0) 

'Tesla V100-SXM2-32GB'

In [ ]:
DATASET_SIZE = 53914
MAX_LEN = 500
SEED = 520
bs = 1
MAX_N_SPLITS = 32

In [ ]:
output_dir = "./biobert_v1.1_pubmed/"
tokenizer = BertTokenizer.from_pretrained(output_dir, do_lower_case=True)  # Add specific options if needed

I0505 14:51:47.145000 47349498433216 tokenization_utils.py:420] Model name './biobert_v1.1_pubmed/' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, bert-base-finnish-cased-v1, bert-base-finnish-uncased-v1, bert-base-dutch-cased). Assuming './biobert_v1.1_pubmed/' is a path, a model identifier, or url to a directory containing tokenizer files.
I0505 14:51:47.147556 47349498433216 tokenization_utils.py:449] Didn't find file ./biobert_v1.1_pubmed/added_tokens.json. We won't load it.
I0505 14:51:47.148517 47349498433216 tokenization_utils.py:449] Did

In [ ]:
print('Loading training data')
train_data_processor = DataProcessor('train',size=None,tokenizer=tokenizer)
tags_vals = train_data_processor.get_tags_vals()

Loading training data


In [ ]:
#Make sure no paragraph has only 'O' tags
paragraphs_tags =  train_data_processor.paragraph_tags
inp = None
for i, paragraph_tags in enumerate(paragraphs_tags):
    assert not all(p == 'O' for p in paragraph_tags)

In [ ]:
#tr_inputs,tr_masks,tr_tags,tr_ttids,train_tokenizer = train_data_processor.get_processed_data()
#print('Loaded training data')

In [ ]:
#train_data_processor.maxN = 32, 32 * 424 = 13568

In [ ]:
print('Loading dev data')
valid_data_processor = DataProcessor('dev',size=None,tokenizer=tokenizer)

Loading dev data


In [ ]:
#val_inputs,val_masks,val_tags,val_ttids,train_val_tokenizer = valid_data_processor.get_processed_data()
#print('Loaded dev data')
#valid_data_processor.maxN = 19

In [ ]:
#tr_inputs = torch.tensor(tr_inputs)
#tr_tags = torch.tensor(tr_tags)
#tr_masks = torch.tensor(tr_masks)
#tr_ttids = torch.tensor(tr_ttids)
#val_inputs = torch.tensor(val_inputs)
#val_tags = torch.tensor(val_tags)
#val_masks = torch.tensor(val_masks)
#val_ttids = torch.tensor(val_ttids)

In [ ]:
torch.cuda.empty_cache() 

In [ ]:
train_dataloader = DataLoader(train_data_processor, shuffle=True, batch_size=bs, collate_fn=train_data_processor.my_collate)

valid_dataloader = DataLoader(valid_data_processor, shuffle=True, batch_size=bs, collate_fn=train_data_processor.my_collate)
print('Created train and dev data loaders')

Created train and dev data loaders


In [ ]:
!nvidia-smi

Tue May  5 14:54:38 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:18:00.0 Off |                    0 |
| N/A   44C    P0    45W / 300W |     11MiB / 32510MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
#BertEmbed = BertModel.from_pretrained("bert-base-uncased", num_labels=len(tags_vals))


#output_dir = "./models/"
# Step 2: Re-load the saved model and vocabulary

# Example for a Bert model
#model = BertForTokenClassification.from_pretrained(output_dir,num_labels=len(tags_vals))


#print('Loaded BERT')

In [ ]:
HIDDEN_SIZE = 768
KERNEL_SIZE = 21
PADDING = (KERNEL_SIZE - 1) // 2
import torch.nn as nn
model = BertModel.from_pretrained("bert-base-uncased",num_labels=len(tags_vals))
class myNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.l = nn.Conv1d(HIDDEN_SIZE,len(tags_vals),kernel_size = KERNEL_SIZE, padding = PADDING)
        #perform softmax on classes dimension
        #self.s = nn.Softmax(dim=1)
    def forward(self,last_hidden_state):
        last_hidden_state = last_hidden_state.transpose(1,2)
        logits = self.l(last_hidden_state)
        #probs = self.s(logits)
        return logits
mynet = myNetwork()

I0505 14:54:38.983725 47349498433216 configuration_utils.py:283] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/akobtan/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0505 14:54:38.986228 47349498433216 configuration_utils.py:319] Model config BertConfig {
  "_num_labels": 3,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": 

In [ ]:
model.cuda();
mynet.cuda();
print('Pushed BERT and Convolution to GPU')

Pushed BERT and Convolution to GPU


In [ ]:
!nvidia-smi

Tue May  5 14:54:44 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:18:00.0 Off |                    0 |
| N/A   45C    P0    59W / 300W |   1472MiB / 32510MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters()) + list(mynet.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [ ]:
from seqeval.metrics import f1_score
from seqeval.metrics import classification_report
from torch.nn import CrossEntropyLoss

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
from transformers import get_linear_schedule_with_warmup
print('Initiating training')
closs = CrossEntropyLoss(ignore_index=-100)
epochs = 4
max_grad_norm = 1.0
F_scores = np.zeros(epochs,float)
patience = 6
accumulation_steps = 16
# Total number of training steps is number of batches * number of epochs.
total_steps = (len(train_dataloader) / accumulation_steps) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

for e in trange(epochs):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    predictions , true_labels = [], []
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        for split in range(len(batch[0])):
            b_input_ids = batch[0].pop(0).to(device)
            b_input_mask = batch[1].pop(0).to(device)
            b_labels = batch[2].pop(0).to(device)
            b_ttids = batch[3].pop(0).to(device)
            # forward pass
            # loss = model(b_input_ids, token_type_ids=b_ttids, attention_mask=b_input_mask, labels=b_labels)
            if split == 0:
                last_hidden_state = model(b_input_ids, token_type_ids=b_ttids, attention_mask=b_input_mask)[0]
            else:
                last_hidden_state = torch.cat([last_hidden_state,model(b_input_ids, token_type_ids=b_ttids, attention_mask=b_input_mask)[0]], dim = 1)
                b_labels = torch.cat([prev_labels,b_labels],dim=1)
                b_input_ids = torch.cat([prev_ids, b_input_ids], dim = 1)
            del b_input_mask, b_ttids
            torch.cuda.empty_cache()
            prev_labels = copy.copy(b_labels)
            prev_ids = b_input_ids
        del prev_labels, prev_ids
        torch.cuda.empty_cache()
        b_logits = mynet(last_hidden_state)
        del last_hidden_state
        b_logits = b_logits.transpose(1,2)
        logits = b_logits.view(-1,len(tags_vals))
        labels = b_labels.view(-1)
        if len(logits) != len(labels):
            print(b_input_ids)
            print(train_data_processor.tokenizer.convert_ids_to_tokens(b_input_ids[0]))
            print(b_input_ids.size())
            print('Logits:')
            print(logits)
            print(logits.size())
            print('Labels:')
            print(labels)
            print(labels.size())
            exit()
        loss = closs(logits, labels)
        # backward pass
        del logits, labels
        torch.cuda.empty_cache()
        b_logits = b_logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(b_logits, axis=2)])
        true_labels.append(label_ids)
        del b_logits, label_ids, b_labels
        torch.cuda.empty_cache()
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += 1
        nb_tr_steps += 1
        if (step+1)%accumulation_steps == 0:
            # gradient clipping
            torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
            # update parameters
            optimizer.step()
            optimizer.zero_grad()
            # Update the learning rate.
            scheduler.step()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags_vals[l_ii] if l_ii!=-100 else '[PAD]' for l in true_labels for l_i in l for l_ii in l_i]
    c_f1 = f1_score(valid_tags, pred_tags)
    rep = classification_report(valid_tags, pred_tags)
    print('Training classification report:',rep)
    print("Training F1-Score: {}".format(c_f1))
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        with torch.no_grad():
            # add batch to gpu
            for split in range(len(batch[0])):
                b_input_ids = batch[0].pop(0).to(device)
                b_input_mask = batch[1].pop(0).to(device)
                b_labels = batch[2].pop(0).to(device)
                b_ttids = batch[3].pop(0).to(device)
               # forward pass
                # loss = model(b_input_ids, token_type_ids=b_ttids, attention_mask=b_input_mask, labels=b_labels)
                if split == 0:
                    last_hidden_state = model(b_input_ids, token_type_ids=b_ttids, attention_mask=b_input_mask)[0]
                else:
                    last_hidden_state = torch.cat([last_hidden_state,model(b_input_ids, token_type_ids=b_ttids, attention_mask=b_input_mask)[0]], dim = 1)
                    b_labels = torch.cat([prev_labels,b_labels],dim=1)
                del b_input_ids, b_input_mask, b_ttids
                torch.cuda.empty_cache()
                prev_labels = copy.copy(b_labels)
            del prev_labels
            torch.cuda.empty_cache()
            b_logits = mynet(last_hidden_state)
            b_logits = b_logits.transpose(1,2)
            logits = b_logits.view(-1,len(tags_vals))
            labels = b_labels.view(-1)
            tmp_eval_loss = closs(logits, labels)
            del logits, labels
            torch.cuda.empty_cache()
        b_logits = b_logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(b_logits, axis=2)])
        true_labels.append(label_ids)
        tmp_eval_accuracy = flat_accuracy(b_logits, label_ids)
        del b_logits, label_ids, b_labels
        torch.cuda.empty_cache()
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        nb_eval_examples += 1
        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags_vals[l_ii] if l_ii!=-100 else '[PAD]' for l in true_labels for l_i in l for l_ii in l_i]
    c_f1 = f1_score(valid_tags, pred_tags)
    rep = classification_report(valid_tags, pred_tags)
    print(rep)
    print("F1-Score: {}".format(c_f1))
    #k epochs no improvement
    F_scores[e] = c_f1
    prev_Fs = np.arange(e-patience,e)
    prev_indices = prev_Fs[prev_Fs>=0]
    if all(F_scores[e] - F_scores[prev_indices] < 0 ) and e > patience:
        print('Done training')
        break
    else:
        print('Still training')
        #save checkpoint

  0%|          | 0/4 [00:00<?, ?it/s]

Initiating training


In [32]:
print('Initiating validation on dev set')
# VALIDATION on validation set
model.eval()
mynet.eval()
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
predictions , true_labels = [], []
for batch in valid_dataloader:
    with torch.no_grad():
        # add batch to gpu
        for split in range(len(batch[0])):
            b_input_ids = batch[0].pop(0).to(device)
            b_input_mask = batch[1].pop(0).to(device)
            b_labels = batch[2].pop(0).to(device)
            b_ttids = batch[3].pop(0).to(device)
           # forward pass
            # loss = model(b_input_ids, token_type_ids=b_ttids, attention_mask=b_input_mask, labels=b_labels)
            if split == 0:
                last_hidden_state = model(b_input_ids, token_type_ids=b_ttids, attention_mask=b_input_mask)[0]
            else:
                last_hidden_state = torch.cat([last_hidden_state,model(b_input_ids, token_type_ids=b_ttids, attention_mask=b_input_mask)[0]], dim = 1)
                b_labels = torch.cat([prev_labels,b_labels],dim=1)
            del b_input_ids, b_input_mask, b_ttids
            torch.cuda.empty_cache()
            prev_labels = copy.copy(b_labels)
        del prev_labels
        torch.cuda.empty_cache()
        b_logits = mynet(last_hidden_state)
        b_logits = b_logits.transpose(1,2)
        logits = b_logits.view(-1,len(tags_vals))
        labels = b_labels.view(-1)
        tmp_eval_loss = closs(logits, labels)
        del logits, labels
        torch.cuda.empty_cache()
    b_logits = b_logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    predictions.extend([list(p) for p in np.argmax(b_logits, axis=2)])
    true_labels.append(label_ids)
    tmp_eval_accuracy = flat_accuracy(b_logits, label_ids)
    del b_logits, label_ids, b_labels
    torch.cuda.empty_cache()
    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy
    nb_eval_examples += 1
    nb_eval_steps += 1
eval_loss = eval_loss/nb_eval_steps
print("Validation loss: {}".format(eval_loss))
print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
valid_tags = [tags_vals[l_ii] if l_ii!=-100 else '[PAD]' for l in true_labels for l_i in l for l_ii in l_i]
c_f1 = f1_score(valid_tags, pred_tags)
rep = classification_report(valid_tags, pred_tags)
print(rep)
print("F1-Score: {}".format(c_f1))


Initiating validation on dev set
Validation loss: 0.05402455055537064
Validation Accuracy: 0.9741180001942487
           precision    recall  f1-score   support

      ans       0.40      0.29      0.34     63389
    [PAD]       0.00      0.00      0.00     67403

micro avg       0.40      0.14      0.21    130792
macro avg       0.19      0.14      0.16    130792

F1-Score: 0.20836896381830433


In [31]:
print('Saving model')
from pytorch_pretrained_bert import WEIGHTS_NAME, CONFIG_NAME

output_dir = "./newmodel_notsure/"

# Step 1: Save a model, configuration and vocabulary that you have fine-tuned

# If we have a distributed model, save only the encapsulated model
# (it was wrapped in PyTorch DistributedDataParallel or DataParallel)
model_to_save = model.module if hasattr(model, 'module') else model

# If we save using the predefined names, we can load using `from_pretrained`
output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
output_config_file = os.path.join(output_dir, CONFIG_NAME)

torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(output_dir)

I0507 10:38:58.924122 47349498433216 modeling.py:230] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


Saving model


('./newmodel_notsure/vocab.txt',)

In [33]:
print('Loading test data')
test_data_processor = DataProcessor('test',size=None,tokenizer=tokenizer)

Loading test data


In [38]:
test_data_processor.paragraphs[0]

"more than meets the eye : the ‘ pink salmon patch ’ summary  ocular adnexal lymphomas account for 1 – 2 % of all non-hodgkin ' s lymphomas . conjunctiva is the primary site of involvement in one - third of cases . we present a case of a 47 - year - old hispanic woman who presented with left eye itching and irritation associated with a painless pink mass . physical examination revealed the presence of a ‘ pink salmon - patch ’ involving her left medial conjunctiva . orbital ct showed a subcentimeter left preseptal soft tissue density . biopsy revealed a dense subepithelial lymphoid infiltrate comprised predominantly of b cells that did not coexpress cd5 or cd43 . these findings were consistent with b - cell marginal zone lymphoma . further staging assessment did not reveal disseminated disease . she had stage 1e extranodal marginal zone lymphoma as per ann arbor staging system . she received external beam radiotherapy to her left eye with complete resolution of the lymphoma in 2 months

In [34]:
torch.cuda.empty_cache()

In [40]:
test_dataloader = DataLoader(test_data_processor, shuffle=True, batch_size=bs, collate_fn=test_data_processor.my_collate)
print('Created train and dev data loaders')

Created train and dev data loaders


In [41]:
!nvidia-smi

Thu May  7 11:39:27 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:18:00.0 Off |                    0 |
| N/A   43C    P0    58W / 300W |   2962MiB / 32510MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [42]:
print('Initiating testing on test set')
# Testing on test set
model.eval()
mynet.eval()
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
predictions , true_labels = [], []
for batch in test_dataloader:
    with torch.no_grad():
        # add batch to gpu
        for split in range(len(batch[0])):
            b_input_ids = batch[0].pop(0).to(device)
            b_input_mask = batch[1].pop(0).to(device)
            b_labels = batch[2].pop(0).to(device)
            b_ttids = batch[3].pop(0).to(device)
           # forward pass
            # loss = model(b_input_ids, token_type_ids=b_ttids, attention_mask=b_input_mask, labels=b_labels)
            if split == 0:
                last_hidden_state = model(b_input_ids, token_type_ids=b_ttids, attention_mask=b_input_mask)[0]
            else:
                last_hidden_state = torch.cat([last_hidden_state,model(b_input_ids, token_type_ids=b_ttids, attention_mask=b_input_mask)[0]], dim = 1)
                b_labels = torch.cat([prev_labels,b_labels],dim=1)
            del b_input_ids, b_input_mask, b_ttids
            torch.cuda.empty_cache()
            prev_labels = copy.copy(b_labels)
        del prev_labels
        torch.cuda.empty_cache()
        b_logits = mynet(last_hidden_state)
        b_logits = b_logits.transpose(1,2)
        logits = b_logits.view(-1,len(tags_vals))
        labels = b_labels.view(-1)
        tmp_eval_loss = closs(logits, labels)
        del logits, labels
        torch.cuda.empty_cache()
    b_logits = b_logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    predictions.extend([list(p) for p in np.argmax(b_logits, axis=2)])
    true_labels.append(label_ids)
    tmp_eval_accuracy = flat_accuracy(b_logits, label_ids)
    del b_logits, label_ids, b_labels
    torch.cuda.empty_cache()
    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy
    nb_eval_examples += 1
    nb_eval_steps += 1
eval_loss = eval_loss/nb_eval_steps
print("Test loss: {}".format(eval_loss))
print("Test Accuracy: {}".format(eval_accuracy/nb_eval_steps))
pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
valid_tags = [tags_vals[l_ii] if l_ii!=-100 else '[PAD]' for l in true_labels for l_i in l for l_ii in l_i]
c_f1 = f1_score(valid_tags, pred_tags)
rep = classification_report(valid_tags, pred_tags)
print(rep)
print("F1-Score: {}".format(c_f1))


Initiating testing on test set
Test loss: 0.057050696317094585
Test Accuracy: 0.9731057468293697
           precision    recall  f1-score   support

      ans       0.40      0.29      0.34     67677
    [PAD]       0.00      0.00      0.00     69475

micro avg       0.40      0.14      0.21    137152
macro avg       0.20      0.14      0.17    137152

F1-Score: 0.21097123265655973
